Scrape a confluence FAQ, put results in a postgres database

In [ ]:
%pip install python-dotenv "psycopg[binary,pool]" 

In [82]:
import requests
import json
from dotenv import dotenv_values
from bs4 import BeautifulSoup
from base64 import b64encode
import psycopg
from dataclasses import dataclass,asdict
from psycopg.rows import class_row

In [83]:
@dataclass
class Article:
    id:str
    parent:str
    version:int
    title:str
    status:str
    hasvectors:bool = False
    def dict(self):
        return {k: str(v) for k, v in asdict(self).items()}
    
class ConfluenceClient:
    def __init__(self) -> None:
        config = dotenv_values()
        self._user = config["JIRAUSER"]
        self._passwd = config["JIRAPASS"] 
        self.url = config["JIRABASEURL"]
        self.space=config["ROOTSPACEID"]
        self._sdbaseurl=config["JIRASDBASEURL"]
    def _basicauth(self):
        raw = f'{self._user}:{self._passwd}'.encode()
        return "Basic " + b64encode(raw).decode()
    def _getheaders(self,accept="application/json"):
        return {"Authorization":self._basicauth(),"Accept":accept}
    def convertArticle(self,raw):
        parent = raw["parent"] if "parent" in raw else ""
        version = int(raw["version"]["number"])
        return Article(raw["id"],parent,version,raw["title"],raw["status"])
            
    def getarticle(self,id):
        resp = requests.get(self.wikiUrl(id),headers=self._getheaders("text/html"))
        return resp.content
    def getspace(self,spaceId):
        resp = requests.get(f'{self.url}/wiki/api/v2/spaces/{self.space}',headers=self._getheaders())
        raw = json.loads(resp.content)
        return dict(id=raw["id"],name=raw["name"],key=raw["key"])
    def getspacewiki(self,all=False):
        resp = requests.get(f'{self.url}/wiki/api/v2/spaces/{self.space}/pages',headers=self._getheaders())
        sect = json.loads(resp.content)
        articles = list(map(self.convertArticle,sect["results"]))
        while(all & ('_links' in sect)& ('next' in sect['_links'])):
            resp = requests.get(f'{self.url}{sect["_links"]["next"]}',headers=self._getheaders())
            sect = json.loads(resp.content)
            articles.extend(map(self.convertArticle,sect["results"]))
        return articles
    def portalUrl(self,articleId):
        return f'{self._sdbaseurl}/servicedesk/customer/portal/6/article/{articleId}'
    def wikiUrl(self,articleId):
        if(not hasattr(self,"_spacekey" )):
            self._spacekey = self.getspace(self.space)["key"]
        return f'{self.url}/wiki/spaces/{self._spacekey}/pages/{articleId}'
    def articleText(self,html):
        content = BeautifulSoup(html).find("div",{"class":"ak-renderer-document"})
        if(content!=None): return content.get_text('\n')


In [155]:
class dbClient:
    def __init__(self) -> None:
        self._password = dotenv_values()["POSTGRES_PASSWORD"]
    def connect(self,dbname="ragtest",autocommit=False):
        return psycopg.connect(f'host=pgvector dbname={dbname} user=postgres password={self._password}',autocommit=autocommit)
    
    def selectArticles(self):
        with self.connect() as conn:
            with conn.cursor(row_factory=class_row(Article)) as cur:
                cur.execute("select * from articles")
                return  cur.fetchall()
    def saveArticle(self,article:Article):
        with self.connect() as conn:
            with conn.cursor() as cur:
                cur.execute(""" INSERT INTO articles (id,parent,version,title,status,hasvectors )
                VALUES (%(id)s, %(parent)s, %(version)s,%(title)s, %(status)s,%(hasvectors)s ); """,
                article.dict())
    def saveArticleContents(self,id,content):
        with self.connect() as conn:
            with conn.cursor() as cur:
                cur.execute(""" INSERT INTO articlescontents (id,contents) VALUES (%(id)s, %(content)s); """,
                dict(id=id,content=content))
    def updateDb(self,articles:list[Article]):
        old={}
        toUpdate=[]
        for article in self.selectArticles():
            old[article.id] = article
        for article in articles:
            if not (article.id in old and article.version == old[article.id].version ==article.version):
                toUpdate.append(article)
        with self.connect() as conn:
            with conn.cursor() as cur:
                for article in toUpdate:
                    if article.id in old:
                        cur.execute("DELETE FROM ARTICLES WHERE ID = %s",(article.id,))
                    cur.execute(""" INSERT INTO articles (id,parent,version,title,status,hasvectors )
                    VALUES (%(id)s, %(parent)s, %(version)s,%(title)s, %(status)s,%(hasvectors)s ); """,
                    article.dict())
    def idswithcontents(self):
        with self.connect() as conn:
            with conn.cursor() as cur:
                res = cur.execute("select id from articlescontents").fetchall()
                ids = []
                for r in res:
                    ids.append(r[0])
                return set(ids)
    def content(self,id):
        with self.connect() as conn:
            with conn.cursor() as cur:
                res = cur.execute("select contents from articlescontents where id = %s",(id,)).fetchone()
                return res[0] if res !=None else None
    def createspace(self,id,key):
        with self.connect() as conn:
            with conn.cursor() as cur:
                cur.execute(""" INSERT INTO spaces (id,key) VALUES (%(id)s, %(key)s); """,
                dict(id=id,key=key))
    def createDb(self):
        try:
            with self.connect("postgres",True) as conn:
                with conn.cursor() as cur:
                    cur.execute("create database ragtest;")
        except:
            pass
        with self.connect() as conn:
            with conn.cursor() as cur:
                cur.execute("""
                CREATE TABLE IF NOT EXISTS public.articles (
                    id character varying(20) NOT NULL,
                    parent character varying(20) NOT NULL,
                    version integer NOT NULL,
                    status character varying(20) NOT NULL,
                    hasvectors boolean,
                    title text NOT NULL,
                    CONSTRAINT articles_pkey PRIMARY KEY (id)
                )
                TABLESPACE pg_default;
                """)
                cur.execute("""
                CREATE TABLE IF NOT EXISTS public.articlescontents (
                    id character varying(20) NOT NULL references public.articles(id) ON DELETE CASCADE,
                    contents text NOT NULL,
                    CONSTRAINT articlescontents_pkey PRIMARY KEY (id)
                )
                TABLESPACE pg_default;
                """)
                cur.execute("""
                CREATE TABLE IF NOT EXISTS public.spaces ( id character varying(20) NOT NULL, key text NOT NULL, 
                            CONSTRAINT spaces_pkey PRIMARY KEY (id) ) TABLESPACE pg_default; """)
db = dbClient()

In [149]:
#initialize
db.createDb()
client = ConfluenceClient()
space=client.getspace(client.space)
try:
    db.createspace(space["id"],space["key"])
except:
    pass

In [150]:
# Download article headers
articles = client.getspacewiki(True)
db.updateDb(articles)

In [159]:
# Download article contents
existing = db.idswithcontents() 
for article in db.selectArticles():
    if not article.id in existing:
        contents = client.articleText(client.getarticle(article.id))
        db.saveArticleContents(article.id,contents)
